In [37]:
!pip install kaggle

In [38]:
#configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [39]:
!kaggle datasets list -s 'Restaurant_Reviews.tsv  '

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

data = pd.read_csv('Restaurant_Reviews.tsv',sep='\t')
data.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [41]:
data.shape

(1000, 2)

In [42]:
data['char_count'] = data['Review'].apply(len)
data.head()

,Review,Liked,char_count
0,Wow... Loved this place.,1,24
1,Crust is not good.,0,18
2,Not tasty and the texture was just nasty.,0,41
3,Stopped by during the late May bank holiday of...,1,87
4,The selection on the menu was great and so wer...,1,59


In [43]:
data['word_count'] = data['Review'].apply(lambda x:len(x.split()))
data.head()

,Review,Liked,char_count,word_count
0,Wow... Loved this place.,1,24,4
1,Crust is not good.,0,18,4
2,Not tasty and the texture was just nasty.,0,41,8
3,Stopped by during the late May bank holiday of...,1,87,15
4,The selection on the menu was great and so wer...,1,59,12


In [44]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [45]:
data['sent_count'] = data['Review'].apply(lambda x : len(nltk.sent_tokenize(str(x))))
data.head()

,Review,Liked,char_count,word_count,sent_count
0,Wow... Loved this place.,1,24,4,2
1,Crust is not good.,0,18,4,1
2,Not tasty and the texture was just nasty.,0,41,8,1
3,Stopped by during the late May bank holiday of...,1,87,15,1
4,The selection on the menu was great and so wer...,1,59,12,1


In [46]:
#char count for positive review
data[data['Liked']==1]['char_count'].mean()

np.float64(55.88)

In [47]:
#char count for negative review
data[data['Liked']==0]['char_count'].mean()

np.float64(60.75)

In [48]:
import re
data['Review'][1]


'Crust is not good.'

In [49]:
review = re.sub('[^a-zA-Z]',' ',data['Review'][1]).lower().split()
display(review)

['crust', 'is', 'not', 'good']

In [50]:
from nltk.corpus import stopwords
import nltk
# Download stopwords if not already downloaded
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

# Example of removing stopwords from the processed review
review = re.sub('[^a-zA-Z]',' ',data['Review'][1]).lower().split()
review = [word for word in review if not word in set(stopwords.words('english'))]
display(review)

['crust', 'good']

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Apply TfidfVectorizer to the 'Review' column
vectorizer = TfidfVectorizer(max_features=1500)  # You can adjust max_features
X = vectorizer.fit_transform(data['Review']).toarray()

# Display the shape of the vectorized data
print("Shape of vectorized data:", X.shape)

Shape of vectorized data: (1000, 1500)


In [52]:
# 'Liked' column is the target variable
y=data['Liked']
print(y)

0      1
1      0
2      0
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: Liked, Length: 1000, dtype: int64


In [53]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% train, 20% test

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (800, 1500)
Shape of X_test: (200, 1500)
Shape of y_train: (800,)
Shape of y_test: (200,)


In [54]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score


# Initialize and train the Naive Bayes model
nb = GaussianNB()
nb .fit(X_train, y_train)

# Make predictions on the test set
y_pred =nb.predict(X_test)

# Evaluate the model
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))

Naive Bayes Accuracy: 0.68


In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Initialize and train the logistic regression  classifier
lr= LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))

Logistic Regression Accuracy: 0.82


In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf= RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# Evaluate the model
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))

Random Forest Accuracy: 0.765


In [57]:
import joblib
joblib.dump(rf,'Restaurant _review_model')

['Restaurant _review_model']

# New section